In [1]:
# Python
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import mysql
import mysql.connector
import pymysql

In [2]:
query = f"""SELECT fecha,tmed,
tmin, tmax
FROM valores_climatologicos
WHERE
YEAR(fecha)
BETWEEN 2020 AND 2024
"""

In [3]:
database = "AEMET"

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "nueva_contraseña",
                             database = database)

cursor = db.cursor()

cursor.execute(query) # Ejecutamos la query
column_names = cursor.column_names # Nombre de las columnas de la tabla

# Guardamos los datos de la tabla
data = cursor.fetchall()


df = pd.DataFrame(data, columns = column_names)

cursor.close()
db.close()

In [4]:
df = df[["fecha", "tmed"]]
df

,fecha,tmed
0,2013-01-01,8.1
1,2013-01-01,7.0
2,2013-01-01,4.9
3,2013-01-01,8.8
4,2013-01-01,13.1
...,...,...
7336000,2024-11-01,13.2
7336001,2024-11-01,NaN
7336002,2024-11-01,NaN
7336003,2024-11-01,17.0


In [5]:
df = df[["fecha", "tmed"]]
df.rename(columns = {"fecha" : "ds",
                     "tmed" : "y"}, inplace=True)
df['ds'] = pd.to_datetime(df['ds'])

In [ ]:
#MODELO

m = Prophet(interval_width=0.95)
m.fit(df)

19:36:08 - cmdstanpy - INFO - Chain [1] start processing


In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()



In [ ]:


# Graficar usando Prophet
fig1 = m.plot(forecast)

# Personalizar los ejes
ax = fig1.gca()  # Obtener los ejes actuales de la figura
ax.set_xlim(['2024-01-01', '2025-09-30'])  # Cambiar límites del eje x (fechas)
ax.set_ylim([0, 50])  # Cambiar límites del eje y (valores)

# Etiquetas personalizadas
ax.set_xlabel('Fecha', fontsize=12)
ax.set_ylabel('Valor', fontsize=12)

# Mostrar la gráfica
plt.show()